In [2]:
!pip install QRCode
!pip -q install diffusers transformers accelerate torch xformers


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.1/58.1 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 50.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 87.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 MB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 31.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 119.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 86.5 MB/s eta 0:00:00


In [3]:
!pip install requests

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import os
import qrcode
import requests
from PIL import Image
import torch
from diffusers import StableDiffusionControlNetImg2ImgPipeline, ControlNetModel, DDIMScheduler


# Generate a QR code for a given text or URL with the highest error correction level
def generate_qr(data, id):
    filename = f"{id}-qrcode.png"

    qr = qrcode.QRCode(version=1,
                       error_correction=qrcode.constants.ERROR_CORRECT_H,
                       box_size=10,
                       border=4)
    qr.add_data(data)
    qr.make(fit=True)

    # Create an image from the QR code
    qr_image = qr.make_image(fill_color="black", back_color="white")

    # Save the image file as PNG format
    qr_image.save(filename, "PNG")


# Function to resize an image to a given resolution without adding padding
def resize_image(input_image: Image, resolution: int) -> Image:
    input_image = input_image.convert("RGB")
    resized_image = input_image.resize((resolution, resolution), resample=Image.LANCZOS)
    return resized_image


# Function to download an image from a URL and save it as a PNG file
def download_image(image_url, save_path):
    try:
        response = requests.get(image_url, stream=True)
        response.raise_for_status()

        with open(save_path, 'wb') as file:
            for chunk in response.iter_content(chunk_size=8192):
                file.write(chunk)

        # Open the downloaded image using PIL
        image = Image.open(save_path)

        # Save the image as PNG format
        image.save(save_path, "PNG")

        return True

    except requests.exceptions.RequestException as e:
        print(f"Error downloading image: {e}")
        return False


# Function to download the model and get the pipeline
def get_pipeline():
    try:
        # Load the ControlNet model from a pretrained checkpoint
        controlnet = ControlNetModel.from_pretrained(
            "DionTimmer/controlnet_qrcode-control_v11p_sd21",
            torch_dtype=torch.float16)

        # Create a StableDiffusionControlNetImg2ImgPipeline with the loaded ControlNet model
        pipe = StableDiffusionControlNetImg2ImgPipeline.from_pretrained(
            "stabilityai/stable-diffusion-2-1",
            controlnet=controlnet,
            safety_checker=None,
            torch_dtype=torch.float16)

        # Enable memory-efficient attention for the pipeline
        pipe.enable_xformers_memory_efficient_attention()

        # Set the scheduler for the pipeline to DDIMScheduler with its current configuration
        pipe.scheduler = DDIMScheduler.from_config(pipe.scheduler.config)

        # Enable CPU offload for the model
        pipe.enable_model_cpu_offload()

        return pipe

    except Exception as e:
        print(f"Error initializing pipeline: {e}")
        return None


In [14]:
def generate_qr_art(id, url, reference_image_url, prompt, pipe):
    try:
        # Generate the QR code image
        generate_qr(url, id)

        # Download the reference image
        reference_image_path = f"{id}-reference.png"
        if not download_image(reference_image_url, reference_image_path):
            return False

        # Load the QR code image from local storage
        source_image_path = f"{id}-qrcode.png"
        source_image = Image.open(source_image_path)
        source_image = resize_image(source_image, 768)
        print(source_image)

        # Load the initial image
        init_image = Image.open(reference_image_path)
        init_image = resize_image(init_image, 768)
        print(init_image)

        generator = torch.manual_seed(123121231)

        # Generate the image using the pipeline
        image = pipe(
            prompt=prompt,
            negative_prompt="ugly, disfigured, low quality, blurry",
            image=init_image,
            control_image=source_image,
            width=768,
             height=768,
             guidance_scale=7.5,
             controlnet_conditioning_scale=0.95,
             generator=generator,
             strength=0.9,
             num_inference_steps=150,
        )

        # Save the generated image
        output_path = "output.png"
        image.images[0].save(output_path)

        # Clean up temporary files
        os.remove(source_image_path)
        os.remove(reference_image_path)

        return True

    except Exception as e:
        print(f"Error generating QR art: {e}")
        return False

# Build Pipeline

In [3]:
pipe = get_pipeline()

# Generate QART

In [ ]:
ref = "https://www.google.com/url?sa=i&url=http%3A%2F%2Ft3.gstatic.com%2Flicensed-image%3Fq%3Dtbn%3AANd9GcRt02L1YRC5s_uF7MMYe-2OQPf7uTlZwIPKHJIGeuRT6KCy0Y-GMGgym4leG1e-gJKR&psig=AOvVaw24itJs9-xKCHGbOQm83Rez&ust=1687164565037000&source=images&cd=vfe&ved=0CBEQjRxqFwoTCNjekve3zP8CFQAAAAAdAAAAABAE"
generate_qr_art('aqib','https://stackoverflow.com/questions/990754/how-to-leave-exit-deactivate-a-python-virtualenv',ref,'A Japanese Samurai',pipe)

<PIL.Image.Image image mode=RGB size=768x768 at 0x7F0A658FF250>
<PIL.Image.Image image mode=RGB size=768x768 at 0x7F0910F58460>


  0%|          | 0/135 [00:00<?, ?it/s]